In [1]:
from vis_gpx import main

In [2]:
from collections import namedtuple
Args = namedtuple('arguments', ['root_dir', 'match_file', 'segs_for_clips_csv', 'segs'])
root_dir = 'test_data/'
match_file = 'gpx_video_match.csv'
segs_for_clips_csv = 'segs_for_clips.csv'
segs = 'segments_dc.geojson'
args = Args(root_dir, match_file, segs_for_clips_csv, segs)

In [3]:
main(args)

# Abnomal examples

In [191]:
gpx = 'Sample Data/GPX/Track_2017-02-21 125536.gpx'
vfile, raw_locs, tms = parse_gpx(gpx)

snapped_locs = snap2road(raw_locs)

gpdf_raw = gp.GeoDataFrame([Point(x) for x in raw_locs], columns=['geometry']).reset_index()
gpdf_raw.geometry = gpdf_raw.buffer(bfr)
gpdf_raw['color'] = '#aaa'
gpdf_snapped = gp.GeoDataFrame([Point(x) for x in snapped_locs], columns=['geometry']).reset_index()
gpdf_snapped.geometry = gpdf_snapped.buffer(bfr)
gpdf_snapped['color'] = '#800'
gpdfs = [gpdf_raw, gpdf_snapped]
binding_data = [['ly1', 'raw'], ['ly2','snapped']]

html_title = 'anomaly1'
file_path = ''
file_name = 'anomaly1'
lon, lat = -77.0908494, 38.9045525  #D.C.
zoom = 12
init_layers = ['streets']
map_layers = ['light','streets', 'satellite']
create_map_visualization(html_title, file_path, file_name, lat, lon, zoom, init_layers, map_layers, binding_data, gpdfs)

In [193]:
gpx = 'Sample Data/GPX/Track_2017-02-21 131933.gpx'
vfile, raw_locs, tms = parse_gpx(gpx)

snapped_locs = snap2road(raw_locs)

gpdf_raw = gp.GeoDataFrame([Point(x) for x in raw_locs], columns=['geometry']).reset_index()
gpdf_raw.geometry = gpdf_raw.buffer(bfr)
gpdf_raw['color'] = '#aaa'
gpdf_snapped = gp.GeoDataFrame([Point(x) for x in snapped_locs], columns=['geometry']).reset_index()
gpdf_snapped.geometry = gpdf_snapped.buffer(bfr)
gpdf_snapped['color'] = '#800'
gpdfs = [gpdf_raw, gpdf_snapped]
binding_data = [['ly1', 'raw'], ['ly2','snapped']]

html_title = 'anomaly2'
file_path = ''
file_name = 'anomaly2'
lon, lat = -77.0908494, 38.9045525  #D.C.
zoom = 12
init_layers = ['streets']
map_layers = ['light','streets', 'satellite']
create_map_visualization(html_title, file_path, file_name, lat, lon, zoom, init_layers, map_layers, binding_data, gpdfs)